In [6]:
import cv2
import numpy as np
import os
from IPython.display import HTML, display
import base64

print("✅ 라이브러리가 준비되었습니다.")

✅ 라이브러리가 준비되었습니다.


In [7]:
import firebase_admin
from firebase_admin import credentials, storage

cred = credentials.Certificate("./serviceAccountKey.json")

if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'storageBucket': 'fairplayfairy-3e2eb.firebasestorage.app'
    })
    print("Firebase 앱이 성공적으로 초기화되었습니다.")
else:
    print("Firebase 앱이 이미 초기화되어 있습니다.")

# Storage 버킷 객체 가져오기 예시
bucket = storage.bucket()
print("Storage 버킷에 접근 성공:", bucket.name)

Firebase 앱이 이미 초기화되어 있습니다.
Storage 버킷에 접근 성공: fairplayfairy-3e2eb.firebasestorage.app


In [8]:
import cv2
import numpy as np
import traceback

def track_aim_in_video(input_path, output_path=None, warmup_frames=0):
    """
    스킵(3초) 로직 제거. WebM 메타 부정확 시에도 전체 프레임 처리.
    warmup_frames: 앞 부분은 계산만 하고 aim_movements에 미저장.
    """
    try:
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            print(f"Error: 영상을 열 수 없습니다. 경로: {input_path}")
            return None

        width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps    = cap.get(cv2.CAP_PROP_FPS)
        try:
            total_frames_raw = cap.get(cv2.CAP_PROP_FRAME_COUNT)
            total_frames = int(total_frames_raw)
            if total_frames <= 0:
                total_frames = None
        except:
            total_frames = None

        if width == 0 or height == 0:
            print("Warning: 해상도 정보 실패")
            cap.release()
            return None

        # 첫 프레임 읽기
        ret, prev_frame = cap.read()
        if not ret:
            print("Error: 첫 프레임 읽기 실패")
            cap.release()
            return None

        roi_x, roi_y = int(width * 0.2), int(height * 0.2)
        roi_w, roi_h = int(width * 0.6), int(height * 0.6)
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        prev_roi  = prev_gray[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]

        aim_movements = []
        out = None
        if output_path:
            use_fps = fps if 1 <= fps <= 480 else 60
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, int(use_fps), (width, height))

        frame_index = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            roi  = gray[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]

            flow = cv2.calcOpticalFlowFarneback(
                prev_roi, roi, None,
                0.5, 3, 15, 3, 5, 1.2, 0
            )
            dx = float(np.mean(flow[...,0]))
            dy = float(np.mean(flow[...,1]))
            aim_dx, aim_dy = -dx, -dy

            if frame_index >= warmup_frames:
                aim_movements.append((aim_dx, aim_dy))

            if out:
                out.write(frame)

            prev_roi = roi
            frame_index += 1

        cap.release()
        if out:
            out.release()
        return aim_movements

    except Exception as e:
        print(f"❌ track_aim_in_video 함수 예외: {e}")
        traceback.print_exc()
        return None

In [9]:
import tempfile
import matplotlib.pyplot as plt

# 영상 리스트 가져오기
videos = list(bucket.list_blobs(prefix='videos/'))
webm_files = [video for video in videos if video.name.endswith('.webm')]

all_videos = webm_files
all_aim_data = []

# 병렬 처리 없이 순차적으로 전체 영상 처리
for i, video_blob in enumerate(all_videos):
    print(f"[{i+1}/{len(all_videos)}] 처리 중: {video_blob.name}")
    fd, temp_path = tempfile.mkstemp(suffix=".webm")
    os.close(fd)
    try:
        video_blob.download_to_filename(temp_path)  # 영상 다운로드
        aim_data = track_aim_in_video(temp_path, output_path=None)
        all_aim_data.append(aim_data)
    except Exception as e:
        print(f"영상 처리 중 오류 발생: {e}")
    finally:
        if os.path.exists(temp_path):
            os.remove(temp_path)

print(f"{len(all_aim_data)}개의 데이터 수집 완료.")


[0/273] 처리 중: videos/01701665-2d22-4a6c-99e0-f36fa91395cf.webm


KeyboardInterrupt: 

In [ ]:
if not all_aim_data:
    print("분석 데이터 없음.")
else:
    # per-video metric(평균)과 velocity 시퀀스 수집
    vel_seqs = []
    per_video_stats = {'vel_mean': [], 'acc_mean': [], 'jerk_mean': [], 'anglechg_mean': []}

    for item in all_aim_data:
        moves = item['data']  # shape (T,2)
        vel = np.linalg.norm(moves, axis=1)
        acc = np.linalg.norm(np.diff(moves, axis=0), axis=1) if len(moves) > 1 else np.array([0.0])
        jerk = np.linalg.norm(np.diff(moves, n=2, axis=0), axis=1) if len(moves) > 2 else np.array([0.0])
        ang = np.arctan2(moves[:,1], moves[:,0]) * 180/np.pi
        angchg = np.abs(((np.diff(ang) + 180) % 360) - 180) if len(ang) > 1 else np.array([0.0])

        per_video_stats['vel_mean'].append(np.mean(vel))
        per_video_stats['acc_mean'].append(np.mean(acc))
        per_video_stats['jerk_mean'].append(np.mean(jerk))
        per_video_stats['anglechg_mean'].append(np.mean(angchg))

        vel_seqs.append(vel)

    # 시간 길이 맞추기 (짧은 시퀀스는 마지막 값으로 패딩)
    max_len = max(len(s) for s in vel_seqs)
    padded = np.zeros((len(vel_seqs), max_len))
    for idx, seq in enumerate(vel_seqs):
        padded[idx, :len(seq)] = seq
        if len(seq) < max_len:
            padded[idx, len(seq):] = seq[-1]

    mean_ts = np.mean(padded, axis=0)
    median_ts = np.median(padded, axis=0)

    # 1) per-video metric boxplot
    plt.figure(figsize=(9,4))
    plt.boxplot([per_video_stats['vel_mean'],
                 per_video_stats['acc_mean'],
                 per_video_stats['jerk_mean'],
                 per_video_stats['anglechg_mean']],
                labels=['Velocity','Acceleration','Jerk','AngleChange'])
    plt.title(f'Per-video metric distributions (n={len(per_video_stats["vel_mean"])})')
    plt.grid(True)
    plt.show()

    # 2) 평균 시간축 시계열
    plt.figure(figsize=(12,4))
    plt.plot(mean_ts, label='Mean Velocity', color='b')
    plt.plot(median_ts, label='Median Velocity', color='g')
    plt.xlabel('Frame')
    plt.ylabel('Velocity (pixels/frame)')
    plt.title('Mean/Median Velocity Time Series (across samples)')
    plt.legend()
    plt.grid(True)
    plt.show()

    # 3) 요약 통계 출력
    for k, vals in per_video_stats.items():
        print(f"{k}: mean={np.mean(vals):.3f}, std={np.std(vals):.3f}")